# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

141b8691f629447f822359b7dafbbb82


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sandnessjoen,66.0217,12.6316,16.29,88,75,2.57,NO,1691166835
1,1,blackmans bay,-43.0167,147.3167,11.07,70,97,6.92,AU,1691166835
2,2,lebu,-37.6167,-73.6500,11.96,55,25,7.08,CL,1691166756
3,3,laurel,39.0993,-76.8483,26.44,77,100,2.06,US,1691166835
4,4,remire-montjoly,4.9167,-52.2667,32.02,66,75,4.63,GF,1691166835


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": city_data_df["Lat"],
    "longitude": city_data_df["Lng"],
    "City": city_data_df["City"],
    "Humidity": city_data_df["Humidity"]
}
# Create a Pandas DataFrame containing coordinates
coordinates_df = pd.DataFrame(coordinates)
# Display sample data
coordinates_df
# Display the map
# Configure the map plot
humidity_map_plot = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)
# Display the map plot
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
# ideal weather = 21 < weather < 28 and humidity < 50
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 28) & (city_data_df["Humidity"] <= 50)]
ideal_weather_df

#null_df = city_data_df[city_data_df["Max Temp"].isnull()]
#null_df
# Drop any rows with null values
final_ideal_weather_df = ideal_weather_df.dropna(subset=["Max Temp", "Humidity"]).reset_index()
final_ideal_weather_df

# Display sample data
final_ideal_weather_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,9,9,masuguru,-11.3667,38.4167,26.05,39,58,4.27,TZ,1691166837
1,82,82,saurimo,-9.6608,20.3916,23.89,23,1,2.52,AO,1691166856
2,95,95,port elizabeth,-33.9180,25.5701,21.16,49,0,1.54,ZA,1691166860
3,111,111,kapoeta,4.7717,33.5903,27.47,47,52,4.15,SS,1691166865
4,143,143,beltran,-27.8291,-64.0610,23.90,37,0,2.73,AR,1691166882


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
copy_df = final_ideal_weather_df.copy()
hotel_details = {
    "City": copy_df["City"],
    "Country": copy_df["Country"],
    "Latitude": copy_df["Lat"],
    "Longitude": copy_df["Lng"],
    "Humidity": copy_df["Humidity"]
}
hotel_df = pd.DataFrame(hotel_details)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,masuguru,TZ,-11.3667,38.4167,39,
1,saurimo,AO,-9.6608,20.3916,23,
2,port elizabeth,ZA,-33.9180,25.5701,49,
3,kapoeta,SS,4.7717,33.5903,47,
4,beltran,AR,-27.8291,-64.0610,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
# Set the parameters for the type of search
limit = 20
radius = 10000
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "limit":limit,
    "apiKey":geoapify_key    
}
# Print a message to follow up the hotel search
print("Starting hotel search")
print("---------------------------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    #pprint(params)
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #pprint(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Indicate that Data Loading is complete 
print("---------------------------------------")
print("Data Retrieval Complete      ")
print("---------------------------------------")

# Display sample data
hotel_df.head()

Starting hotel search
---------------------------------------
masuguru - nearest hotel: No hotel found
saurimo - nearest hotel: Complexo Hoteleiro de Saurimo
port elizabeth - nearest hotel: Waterford Hotel
kapoeta - nearest hotel: Kuleu
beltran - nearest hotel: No hotel found
bardsir - nearest hotel: No hotel found
zbaszyn - nearest hotel: Hotel Senator
patterson - nearest hotel: Hampton Inn & Suites
tabora - nearest hotel: Alpho hotel
susanville - nearest hotel: SureStay Plus Hotel By Best Western Susanville
termas de rio hondo - nearest hotel: Hotel Panamericano
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
newman - nearest hotel: No hotel found
la sarre - nearest hotel: No hotel found
moyale - nearest hotel: Paul’s guest house
caala - nearest hotel: No hotel found
mafinga - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
howard springs - nearest hotel: No hotel found
uige - nearest hotel: No hotel found
opuwo - nearest hotel: Ohakane Lodge
adrasmon - near

,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,masuguru,TZ,-11.3667,38.4167,39,No hotel found
1,saurimo,AO,-9.6608,20.3916,23,Complexo Hoteleiro de Saurimo
2,port elizabeth,ZA,-33.9180,25.5701,49,Waterford Hotel
3,kapoeta,SS,4.7717,33.5903,47,Kuleu
4,beltran,AR,-27.8291,-64.0610,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_coordinates = {
    "latitude": hotel_df["Latitude"],
    "longitude": hotel_df["Longitude"],
    "City": hotel_df["City"],
    #"Humidity": city_data_df["Humidity"]
    "Hotel Name": hotel_df["Hotel Name"],
    "Country": hotel_df["Country"]
}
# Create a Pandas DataFrame containing coordinates
hotel_coordinates_df = pd.DataFrame(hotel_coordinates)
# Display sample data
hotel_coordinates_df
# Display the map
# Configure the map plot
hotel_map_plot = hotel_coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    #size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
)
# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Country,Hotel Name)